## Customer Churn Prediction - Vodafone Corporation

### `Business Understanding`

`Introduction:` Vodafone Corporation is the largest pan-European and African telecoms company. They provide mobile and fixed services to over 300 million customers in 17 countries, partner with mobile networks in 45 more and have one of the world’s largest IoT platforms. Vodafone purpose is to connect for a better future by using technology to improve lives, businesses and help progress inclusive sustainable societies.
Vodafone Corporation faces the ubiquitous challenge of customer churn. Churn, the phenomenon where customers discontinue their services with the company, poses significant financial implications and threatens customer satisfaction. To mitigate this challenge and proactively retain valuable customers, Vodafone seeks to implement a Customer Churn Prediction system. This system aims to identify customers at risk of churning based on their behavioral patterns and characteristics, enabling targeted retention strategies.

`Goal`: The goal of this project is to develop a predictive model capable of accurately identifying customers who are likely to churn from Vodafone's services. By leveraging historical customer data encompassing demographics, usage patterns, subscription history, and churn status, the model must forecast the likelihood of churn for individual customers within a specified time frame. The predicted churn probabilities will enable Vodafone to prioritize retention efforts and deploy targeted interventions to retain at-risk customers, thereby reducing churn rates and enhancing overall customer satisfaction.

`Null Hypothesis:`There is a correlation between the frequency of customer interactions with Vodafone's services and their likelihood of churning.

`Alternative Hypothesis:`There is no significant correlation between the frequency of customer interactions with Vodafone's services and their likelihood of churning

#### Analytical Questions
1. What are the top 3 features that have strong correlation to churn?
2. Does gender have a higher churn rate?
3. 


#### Expected Outcomes
* A predictive model capable of accurately forecasting customer churn probabilities.
* Insights into key drivers and factors influencing churn behavior within Vodafone's customer base.
* Enhanced ability to proactively identify at-risk customers and deploy targeted retention strategies to mitigate churn

## `Data Understanding`

`Import Packages`

In [3]:
# Import packages
import numpy as np
import pandas as pd
import os
from dotenv import dotenv_values, find_dotenv
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import warnings

# remove display limit
pd.set_option('display.max_columns', None)

# hide warnings
warnings.filterwarnings('ignore')

print('Packages imported')


Packages imported


`Extract Datasets`

In [4]:
# Extract datasets from the database
# Load environment variables
environment_variables = dotenv_values(find_dotenv('.env'))

# Get database credentials from .env file
database = environment_variables.get('DATABASE')
server = environment_variables.get('SERVER')
username = environment_variables.get('USERNAME')
password = environment_variables.get('PASSWORD')

# Authenticate connection to the db
conn_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};MARS_Connection=yes;MinProtocolVersion=TLSv1.2;"

# Use the connect method of the pyodbc library and pass in the connection string.
# This will connect to the server and might take a few seconds to be complete. 
# Check your internet connection if it takes more time than necessary
conn = pyodbc.connect(conn_string)

print('Succeeded!')

Succeeded!


In [5]:
# Extract the first dataset from the sql server - df1
query = '''
        SELECT *
        FROM dbo.LP2_Telco_churn_first_3000
        '''
df1 = pd.read_sql_query(query, conn)
df1

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,False,True,False,1,False,None,DSL,False,True,False,False,False,False,Month-to-month,True,Electronic check,29.850000,29.850000,False
1,5575-GNVDE,Male,False,False,False,34,True,False,DSL,True,False,True,False,False,False,One year,False,Mailed check,56.950001,1889.500000,False
2,3668-QPYBK,Male,False,False,False,2,True,False,DSL,True,True,False,False,False,False,Month-to-month,True,Mailed check,53.849998,108.150002,True
3,7795-CFOCW,Male,False,False,False,45,False,None,DSL,True,False,True,True,False,False,One year,False,Bank transfer (automatic),42.299999,1840.750000,False
4,9237-HQITU,Female,False,False,False,2,True,False,Fiber optic,False,False,False,False,False,False,Month-to-month,True,Electronic check,70.699997,151.649994,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2209-XADXF,Female,False,False,False,1,False,None,DSL,False,False,False,False,False,False,Month-to-month,False,Bank transfer (automatic),25.250000,25.250000,False
2996,6620-JDYNW,Female,False,False,False,18,True,True,DSL,True,False,True,False,False,False,Month-to-month,True,Mailed check,60.599998,1156.349976,False
2997,1891-FZYSA,Male,True,True,False,69,True,True,Fiber optic,False,True,False,False,True,False,Month-to-month,True,Electronic check,89.949997,6143.149902,True
2998,4770-UEZOX,Male,False,False,False,2,True,False,Fiber optic,False,True,False,False,False,False,Month-to-month,True,Electronic check,74.750000,144.800003,False


In [9]:
# Load the second dataset from a github repository - df2
df2 = pd.read_csv('https://github.com/Azubi-Africa/Career_Accelerator_LP2-Classifcation/blob/main/LP2_Telco-churn-second-2000.csv?raw=true')
df2.head()

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
# # load the test dataset
# dftest = pd.read_excel("Data/Telco-churn-last-2000.xlsx")
# dftest.head()